This is a notebook to generate all the prompts and seeds in the prompts google sheet

In [1]:
song_name = 'spacetrain_1024' #@param {type:"string"}
res_height = 576 #@param
res_width = 1024 #@param

import os
import pandas as pd
from google.colab import drive

drive.mount('/content/gdrive', force_remount=True)

code_folder = '/content/gdrive/MyDrive/AI-Art Lee'


# fp = os.path.join(code_folder, 'input_data.xlsx')
# df_prompt = pd.read_excel(fp, 'prompts_{}'.format(song_name), index_col=0).dropna(how='all')
# df_prompt

fp = os.path.join(code_folder, song_name, 'prompt_image_definitions.csv')
df_prompt = pd.read_csv(fp, index_col=0).dropna(how='all')


Mounted at /content/gdrive


In [2]:
%%capture
!pip install diffusers
!pip install transformers scipy ftfy accelerate

In [3]:
import torch
from diffusers import StableDiffusionPipeline

pipe = StableDiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-2-1",
                                               torch_dtype=torch.float16,
                                               safety_checker=None
                                               )


pipe = pipe.to("cuda")



Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

# Iterate through prompts and seeds, outputting an image for both

In [4]:
for name, row in df_prompt.iterrows():
  seeds = row['seeds'].split(',')
  seeds = [s.strip() for s in seeds]
  print(seeds)

['1251698433445435', '2287259771456094', '919588236569255', '1345600503815601', '5232148067843148', '1780283600179318', '1590938731344866', '7514589968144458', '5953390400565685', '2805381572470346']
['7513550102712487', '2019222925868596', '6484628732178012', '5802207170251508', '8058345109409010', '6607177028541266', '8205029522770608', '3548472256980440', '2374342233491158', '6706329503192254']
['6548786446951153', '6044285832786262', '8987207457072248', '5588976376677084', '1535333058646368', '5556416826361356', '6466065110328318', '7939248234480387']
['5323038179106160', '2451218105467736', '4376065481117000', '8044106513453894', '3259733038202572', '5653024069654344', '3970471588846124']
['4100634228657733', '7434810728258836', '7160155772081227', '533433906723336', '6596200606858235']
['7634831133439920', '7383671262058718', '5138513910725795', '151854021752774', '4290372915167282', '4249536118218092']
['1348043327693785', '8663222531592233']
['6982753169366518', '25012168779201

In [ ]:
device = "cuda"
generator = torch.Generator(device=device)


#TODO: replace below
width = res_width
height = res_height

skip_existing = True

output_folder = os.path.join(code_folder, song_name, 'prompt_images')

if not os.path.exists(output_folder): os.makedirs(output_folder)

for name, row in df_prompt.iterrows():
  seeds = row['seeds'].split(',')
  seeds = [s.strip() for s in seeds]

  prompt = row['prompt']
  guidance_scale = float(row['guidance_scale'])

  for seed in seeds:
    output_fn = "{}_{}.png".format(name, seed)

    if os.path.exists(os.path.join(output_folder, output_fn)):
      if skip_existing:
        print("{} already exists, skipping".format(output_fn))
        continue

    generator.manual_seed(int(seed))

    latent = torch.randn(
        (1, pipe.unet.in_channels, height // 8, width // 8),
        generator = generator,
        device = device
    )

    with torch.autocast(device):
      images = pipe(
          prompt,
          guidance_scale=guidance_scale,
          latents = latent,
          width=width,
          height=height
      )

    output_image = images.images[0]

    output_image.save(os.path.join(output_folder, output_fn))

<ipython-input-5-59c3de15787f>:33: FutureWarning: Accessing config attribute `in_channels` directly via 'UNet2DConditionModel' object attribute is deprecated. Please access 'in_channels' over 'UNet2DConditionModel's config object instead, e.g. 'unet.config.in_channels'.
  (1, pipe.unet.in_channels, height // 8, width // 8),


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]